# **EDA up_event , down_event variable**

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
# feature_engineering
import re

# ploting
import matplotlib.pyplot as plt
# model initial:
import statsmodels.api as sm

# 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn.cluster import KMeans
#testing:
import unittest

In [2]:
#Set import folder
input_folder = "kaggle/input/linking-writing-processes-to-writing-quality/"

In [3]:
# Load train logs
train_logs = pd.read_csv(input_folder + "train_logs.csv",delimiter = ",",header = 0)
# Set the id and event id as index:
train_logs_indexed = train_logs.set_index(['id', 'event_id'])
# Load train scores
train_scores = pd.read_csv(input_folder +"train_scores.csv", delimiter = ",", header = 0)
scores = pd.Series(data = train_scores['score'].values, index = train_scores['id'].values, name = 'score')

In [4]:
train_logs.head(6)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
5,001519c8,6,107296,107400,104,Input,q,q,q,3,1


In [5]:
# test if the ids are in sync:
n_test = train_logs.groupby('id')['event_id'].sum()
np.sum(n_test.index == scores.index)

2471

In [6]:
n_test.shape

(2471,)

In [7]:
train_logs.tail(3)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240
8405897,fff05981,3619,2070065,2070133,68,Input,.,.,.,1029,240


In [8]:
train_logs.shape

(8405898, 11)

In [9]:
train_logs.up_event.describe()

count     8405898
unique        130
top             q
freq      5364789
Name: up_event, dtype: object

In [10]:
score_60 = scores[scores == 6.0].index.values
score_05 = scores[scores == 0.5].index.values


In [11]:
train_logs.up_event.unique()

array(['Leftclick', 'Shift', 'q', 'Space', 'Backspace', '.', ',', 'Enter',
       'ArrowLeft', "'", ';', 'ArrowRight', '-', '?', 'Tab', '"',
       'ArrowUp', 'ArrowDown', 'm', 'Rightclick', 'i', 'o', 't', '=', 'a',
       'CapsLock', 'Control', 'c', 'v', '/', 'Delete', ':', 'z', '[', '$',
       '(', ')', '+', 'Home', 'End', '\\', 'Meta', '*', '&',
       'AudioVolumeMute', 'x', '!', 'Insert', 'MediaPlayPause', 'w',
       'NumLock', '%', 'V', 'b', '>', 'Alt', 'AudioVolumeUp',
       'ContextMenu', 'AudioVolumeDown', 'n', 'e', '<', 'PageDown', ']',
       'Middleclick', '@', 'F12', 'u', 'j', 's', '\x96', 'Dead', 'y', '{',
       'ScrollLock', '¿', 'p', 'Process', '}', 'MediaTrackPrevious',
       'MediaTrackNext', 'F3', '^', 'Unidentified', 'Cancel', 'h', '2',
       'd', 'r', '`', '\x9b', 'f', 'g', '#', '~', 'PageUp', 'l', 'T', 'A',
       'S', 'ModeChange', '_', 'Escape', 'F11', 'Unknownclick',
       'AltGraph', 'F10', 'F15', 'Clear', 'OS', 'C', 'Ä±', 'M', '|',
       'â\x80\x93', 

In [12]:
train_logs.down_event.unique()

array(['Leftclick', 'Shift', 'q', 'Space', 'Backspace', '.', ',', 'Enter',
       'ArrowLeft', "'", ';', 'ArrowRight', '-', '?', 'Tab', '"',
       'ArrowUp', 'ArrowDown', 'Rightclick', '=', 'CapsLock', 'Control',
       'c', 'v', '/', 'Delete', ':', 'z', '[', '$', '(', ')', '+', 'Home',
       'End', '\\', 'Meta', '*', '&', 'AudioVolumeMute', 'x', '!',
       'Insert', 'MediaPlayPause', 'NumLock', '%', 'V', '>', 'Alt',
       'AudioVolumeUp', 'ContextMenu', 'AudioVolumeDown', 'a', '<',
       'PageDown', ']', 'Middleclick', '@', 'F12', 'j', '\x96', 'Dead',
       't', 's', 'n', 'y', '{', 'ScrollLock', '¿', 'Process', '}',
       'MediaTrackPrevious', 'MediaTrackNext', 'F3', '^', 'Unidentified',
       'Cancel', '2', 'i', 'd', 'r', 'e', '`', '\x9b', 'm', '#', '~',
       'PageUp', 'T', 'A', 'b', 'S', 'ModeChange', '_', 'Escape', 'F11',
       'Unknownclick', 'AltGraph', 'F10', 'h', 'F15', 'Clear', 'OS', 'F',
       'C', 'o', 'Ä±', 'f', 'u', 'w', 'p', 'g', 'M', 'l', '|',
       'â\x80\x

In [13]:
''' 
Eda for Up_event and Down_event comparing if a certain sample plays a music or
has a differnt up and down event 
'''
class EventComboTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self,X, y = None):
        return self

    def transform(self, X):
        feat1 = X.groupby('id')['down_event'].aggregate(lambda x: 1 if("MediaPlayPause" in " ".join(x)) else 0)
        X['diff_up_down'] = X.up_event == X.down_event
        feat2 = X.groupby('id')['diff_up_down'].aggregate(lambda x: 1 if(len(x) == np.sum(x)) else 0)
        output = pd.merge(feat1,feat2, left_index = True, right_index = True)
        
        return output.values 
        
    

In [14]:
''' 
Eda for Up_event  Count Vectorizer
'''
class EventComboTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self,X, y = None):
        return self

    def transform(self, X):
        feat1 = X.groupby('id')['down_event'].aggregate(lambda x: 1 if("MediaPlayPause" in " ".join(x)) else 0)
        X['diff_up_down'] = X.up_event == X.down_event
        feat2 = X.groupby('id')['diff_up_down'].aggregate(lambda x: 1 if(len(x) == np.sum(x)) else 0)
        output = pd.merge(feat1,feat2, left_index = True, right_index = True)
        
        return output.values 
        
    

In [15]:
tx = EventComboTransformer()
result = tx.fit_transform(train_logs)

In [16]:
result

array([[0, 1],
       [0, 1],
       [0, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 0]])

In [17]:
result = train_logs.groupby('id')['up_event'].aggregate(lambda x: " ".join(x))

In [18]:
result

id
001519c8    Leftclick Leftclick Shift q q q q q q Space q ...
0022f953    Leftclick Shift q q q q Space q q Space Shift ...
0042269b    Leftclick Shift q q q q q q q Space q q q Spac...
0059420b    Leftclick Leftclick Shift Shift Shift Shift Sh...
0075873a    Leftclick Shift q q q q q q q q q q q Space q ...
                                  ...                        
ffb8c745    Leftclick Tab Leftclick Space Space Space Spac...
ffbef7e5    Leftclick Leftclick Shift q q q q Space q q q ...
ffccd6fd    Leftclick Leftclick q q q q q q Space q q q q ...
ffec5b38    Leftclick Shift q q q q q q q q Backspace q q ...
fff05981    Leftclick Leftclick q q Space q q q q Space q ...
Name: up_event, Length: 2471, dtype: object

In [19]:
n = CountVectorizer()

In [20]:
n2 = n.fit_transform(result)

In [21]:
n2

<2471x50 sparse matrix of type '<class 'numpy.int64'>'
	with 18921 stored elements in Compressed Sparse Row format>

In [25]:
kk = KMeans(n_clusters = 12, n_init = 20)

In [26]:
kk.fit(n2)

KMeans(n_clusters=12, n_init=20)

In [29]:
z = kk.transform(n2)

In [44]:
z

array([[ 762.60938145,  283.90669468, 2011.43816962, ...,  317.6265668 ,
        2125.58683074,  513.86914112],
       [ 718.05167044,  158.33645965, 1980.30641826, ...,  430.7463053 ,
        2102.26999336,  514.02611767],
       [ 727.77502335,  396.72739202, 1818.7961706 , ...,  254.02043132,
        2122.70626206,  330.29050315],
       ...,
       [1568.34500663, 1561.05468513,  588.8660497 , ..., 1455.83902363,
        2503.36351236, 1085.22127229],
       [ 745.75062714,  203.13308809, 1913.17080006, ...,  402.12444465,
        2116.68836275,  451.81168893],
       [ 803.02650835,  431.27942809, 1989.06264971, ...,  532.45403534,
        2069.50492268,  608.40658282]])

In [34]:
ll = kk.labels_

In [41]:
st.pearsonr(ll, scores.values)

PearsonRResult(statistic=0.445726113513912, pvalue=6.454374984420831e-121)

In [42]:
ll

array([5, 5, 5, ..., 2, 5, 5], dtype=int32)

In [43]:
scores

001519c8    3.5
0022f953    3.5
0042269b    6.0
0059420b    2.0
0075873a    4.0
           ... 
ffb8c745    3.5
ffbef7e5    4.0
ffccd6fd    1.5
ffec5b38    5.0
fff05981    4.0
Name: score, Length: 2471, dtype: float64